# MQP Pipeline


## By Daniel McDonough & Surya Vadivazhagu

This notebook includes all of our scripts into one easy pipeline for easy editing and swapping of components.
Full paper can be viewed at: 

### Download Raw Datasets


Remember to unzip and export the original folder out.


Merge the contents of Movie1_part1 and Movie1_part2 into the same Movie1 folder


[Manning_Simple](https://www.kaggle.com/coffeeoverflow/manning-simple)


[Manning_Movie1](https://www.kaggle.com/coffeeoverflow/manning-movie1-part1)


[Manning_Movie2](https://www.kaggle.com/coffeeoverflow/manning-movie2)




In [66]:
# Imports
import math
import os
import numpy as np
import pandas as pd
from PIL import Image
import mahotas
import cv2
import shutil
import pymrmr
from scipy import ndimage as ndi
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from skimage import exposure
from skimage import morphology
from skimage import color
from skimage.measure import label
from skimage.feature import hog
from skimage.feature import peak_local_max
from skimage.segmentation import clear_border
from skimage.morphology import watershed
from skimage.morphology import extrema
from skimage.transform import resize
import keras
import keras_resnet.models
from keras import backend as K
from keras.utils import to_categorical
from matplotlib import pyplot as plt


### Pre-Processing

#### Prep Workspace

In [17]:
# given a path and foldername, this function checks if it exists already and makes a folder
def makefolder(path,foldername):
    homepath = path+foldername
    # if home path already exists then delete it
    if os.path.isdir(homepath):
        shutil.rmtree(homepath)

    # make the home path
    os.mkdir(homepath)
    print(homepath)
    return homepath


# Make folders required for data storage
def makeFolders(folders = None):
    if folders is None:
        folders = ["./Crops","./LOG","./Gabor","./HOG","./SIFT","./Dataset"]
    for path in folders:
        if os.path.exists(path):
            for root, dirs, files in os.walk(path):
                for file in files:
                    os.remove(os.path.join(root, file))
        else:
            os.mkdir(path)


makefolder("./Organized","/Manning_Simple")
makefolder("./Organized","/Manning_Movie1")
makefolder("./Organized","/Manning_Movie2")
Dataset_location = "./Dataset"
makeFolders()

./Organized/Manning_Simple
./Organized/Manning_Movie1
./Organized/Manning_Movie2


#### Convert Raw Dataset to Overlayed Images

In [18]:
#  make folders based on the Movie named data
def makeMovieFolders(foldername):
    path = "./Organized/"
    homepath = makefolder(path,foldername)
    overlay =makefolder(homepath, "/Overlay")
    return (homepath,green,red,overlay)


# given two images images, combine the red and green channels
def combineImages(redLocation,greenLocation,filename):
    RedImage = cv2.imread(redLocation)
    GreenImage = cv2.imread(greenLocation)

    height, width, layers = RedImage.shape  # height, width, layers of an image
    zeroImgMatrix = np.zeros((height, width), dtype="uint8")  # matrix of zeros (black)

    # The OpenCV image sequence is Blue(B),Green(G) and Red(R)
    (BR, GR, RR) = cv2.split(RedImage)
    (BG, GG, RG) = cv2.split(GreenImage)

    Merge = cv2.merge([zeroImgMatrix, GG,RR ])
    
    cv2.imwrite(filename, Merge)
    
    
# sort files based on the Movie1 Dataset filenames
def sortMovies(name,originalpath):
    (homepath, green, red, overlay) = makeMovieFolders(name)
    cells = os.listdir(originalpath)
    cells.sort()
    for file in cells:
        channel = file[-5]  # channel number of a file
        # if the channel is red
        if channel == str(1):
            basename = file[:-5]  # name of the file without #.tif
            # find the corosponding green image
            greenImageLocation = os.path.join(originalpath,basename+"2.tif")
            if os.path.isfile(greenImageLocation):
           
                # Copy the red file to the red channel
                redImageLocation = os.path.join(originalpath, file)
                
                combinedFilename = os.path.join(overlay,basename+".tif")
                combineImages(red+"/"+basename+"1.tif",green+"/"+basename+"2.tif",combinedFilename)

            else:
                print("This image does not have a corresponding green channel image, skipping...")

# sort files based on the Movie2 Dataset filenames
def sortMovies2(name,originalpath):

    (homepath, green, red, overlay) = makeMovieFolders(name)
    cells = os.listdir(originalpath)
    cells.sort()
    for file in cells:

        channel = file[-5]  # channel number of a file

        # if the channel is red
        if channel == str(2):
            basename = file[:-5]  # name of the file without #.tif

            # find the corosponding green image
            greenImageLocation = os.path.join(originalpath,basename+"1.tif")

            if os.path.isfile(greenImageLocation):
                
                # Copy the red file to the red channel
                redImageLocation = os.path.join(originalpath, file)
                
                combinedFilename = os.path.join(overlay,basename+".tif")
                combineImages(red+"/"+basename+"2.tif",green+"/"+basename+"1.tif",combinedFilename)

            else:
                print("This image does not have a corresponding green channel image, skipping...")

    
# sort files based on the a Dataset with Wells Descriptors
def sortSimple(homepath,organizedpath):
    # in the dataset there are wells
    for well in os.listdir(homepath):
        currdir = os.path.join(homepath,well)
        newfolderpath = makefolder(organizedpath,"/"+well)

        # there are 8 sections in a well (1 to 8)
        for i in range(1,9):
            pos = "XY"+str(i)
            posfolderpath = makefolder(newfolderpath, "/"+pos)
            overlayfolderpath = makefolder(posfolderpath, "/Overlay")

            # get list of all RED images in the directory at this position, in this well
            Redpics = [file for file in os.listdir(currdir) if pos in file and "TxRED.tif" in file]
            # get list of all GREEN images in the directory at this position
            Greenpics = [file for file in os.listdir(currdir) if pos in file and "FITC.tif" in file]
            # Check that there is a corresponding green image to each red image
            for file in Redpics:
                # get equalivalent green channel file name
                base_filename = file.rsplit("_", 1)
                green_filename = base_filename[0] +"_FITC.tif"

                # if equal filename exists...
                if green_filename in Greenpics:

                    #copy green image to organized folder
                    greenImageLocation = os.path.join(currdir, green_filename)
                    
                    # copy red image to the red orgainzed folder
                    redImageLocation = os.path.join(currdir, file)
                    
                    # Overlay the red and green and save it to the overlay image
                    combinedFilename = os.path.join(overlayfolderpath, base_filename[0] + ".tif")
                    combineImages(redImageLocation, greenImageLocation, combinedFilename)

                    
# Sort Raw Movies into overlays                  

originalpath = "./Manning_Simple"
organizedpath = "./Organized/Manning_Simple"                  
sortSimple(originalpath,organizedpath)  # sort by wells method 


# originalpath = "./Manning_Movie1"
# organizedpath = "./Organized/Manning_Movie1"  
# sortMovies(name,originalpath)  # sort by movies method


# originalpath = "./Manning_Movie2"
# organizedpath = "./Organized/Manning_Movie2"  
# sortMovies2(name,originalpath)  # sort by wells method







./Organized/Manning_Simple/Well C3
./Organized/Manning_Simple/Well C3/XY1
./Organized/Manning_Simple/Well C3/XY1/Overlay
./Organized/Manning_Simple/Well C3/XY2
./Organized/Manning_Simple/Well C3/XY2/Overlay
./Organized/Manning_Simple/Well C3/XY3
./Organized/Manning_Simple/Well C3/XY3/Overlay
./Organized/Manning_Simple/Well C3/XY4
./Organized/Manning_Simple/Well C3/XY4/Overlay


KeyboardInterrupt: 

#### Important
From the generated overlays we manually selected 1 image from each overlay folder and placed them into the Dataset folder. This was done to prevent prepresenting a single cell multiple times.


The Dataset folder is used for the rest of the pipeline. 

#### Green Haze removal
Here we do an average subtraction of the green haze.

In [19]:
# Dehaze the green channel
def dehaze(g):
    # find average value
    avg = np.average(g)

    # I - avg
    chosen_sub_a = g - avg

    # make sure no negatives
    chosen_sub_a.astype(int)
    chosen_sub_a = chosen_sub_a.clip(min=0)
    return chosen_sub_a

#### Gamma Correction
Here we do gamma correction on the red channel with y=1.5.

In [20]:
# Adjust the gamma of a given image
def adjust_gamma(r, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    # apply gamma correction using the lookup table
    return cv2.LUT(r, table)

#### Normalize Nuclei Orientation

In [21]:
# Rotate an image given an angle
def rotate_bound(image, angle):
    
    # grab the dimensions of the image and then determine the 
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

#### Upscale Image Quality

In [22]:
# Upscale image   
def bilinear_upscale(image,rate=2):
    rate = 2
    img_linear_x = int(image.shape[1] * rate)
    img_linear_y = int(image.shape[0] * rate)
    pil_im = Image.fromarray(image)
    image = pil_im.resize((img_linear_x, img_linear_y), Image.BILINEAR)  # bilinear interpolation
    image = np.asarray(image)
    return image

#### Pad Image to uniform size

In [23]:
# Pad an image with zeros given the image and the new size
def padImage(img,newhight,newwidth):
    h,w = img.shape
    width_diff = newwidth - w
    height_diff = newhight - h

    th = int(math.floor(height_diff/2))
    bh = int(math.ceil(height_diff / 2))
    rw = int(math.ceil(width_diff / 2))
    lw = int(math.ceil(width_diff / 2))

    new_img = np.pad(img,[(th,bh),(rw,lw)],mode='constant',constant_values=0)
    return new_img


### Feature Detection

#### Get Centroid, Major axis, Minor Axis, Eccentricity, and Rotation angles

In [24]:
# Get Centroid, Major axis, Minor Axis, Eccentricity, and Rotation angles
def get_axis(cnt):

    if len(cnt) < 5:
        return 0,0,0,0,0,0,0

    (x, y), (MA, ma), angle = cv2.fitEllipse(cnt)

    a = ma / 2
    b = MA / 2

    ecc = np.sqrt(a ** 2 - b ** 2) / a

    maj_angle = int(round(angle))
    min_angle = maj_angle - 45
    if min_angle < 0:
        min_angle = min_angle + 180

    return int(round(x)),int(round(y)),int(round(MA)),int(round(ma)),maj_angle,min_angle, ecc



#### Zernlike Moments

In [25]:

# Zernlike Moments
def fd_Zern(img,radius):
    zern = mahotas.features.zernike_moments(img, radius)
    return zern


#### Laplacian of Gaussian of an image

In [26]:
# Laplacian of Gaussian of an image
def fd_LoG(gray_img, sigma=1., kappa=0.75, pad=False):
    """
    Applies Laplacian of Gaussians to grayscale image.

    :param gray_img: image to apply LoG to
    :param sigma:    Gauss sigma of Gaussian applied to image, <= 0. for none
    :param kappa:    difference threshold as factor to mean of image values, <= 0 for none
    :param pad:      flag to pad output w/ zero border, keeping input image size
    """
    assert len(gray_img.shape) == 2
    img = cv2.GaussianBlur(gray_img, (0, 0), sigma) if 0. < sigma else gray_img
    img = cv2.Laplacian(img, cv2.CV_64F)
    rows, cols = img.shape[:2]
    
    # min/max of 3x3-neighbourhoods
    min_map = np.minimum.reduce(list(img[r:rows-2+r, c:cols-2+c]
                                     for r in range(3) for c in range(3)))
    max_map = np.maximum.reduce(list(img[r:rows-2+r, c:cols-2+c]
                                     for r in range(3) for c in range(3)))
    
    # bool matrix for image value positiv (w/out border pixels)
    pos_img = 0 < img[1:rows-1, 1:cols-1]
    
    # bool matrix for min < 0 and 0 < image pixel
    neg_min = min_map < 0
    neg_min[1 - pos_img] = 0
    
    # bool matrix for 0 < max and image pixel < 0
    pos_max = 0 < max_map
    pos_max[pos_img] = 0
    
    # sign change at pixel?
    zero_cross = neg_min + pos_max
    
    # values: max - min, scaled to 0--255; set to 0 for no sign change
    value_scale = 255. / max(1., img.max() - img.min())
    values = value_scale * (max_map - min_map)
    values[1 - zero_cross] = 0.
    
    # optional thresholding
    if 0. <= kappa:
        thresh = float(np.absolute(img).mean()) * kappa
        values[values < thresh] = 0.
    log_img = values.astype(np.uint8)
    if pad:
        log_img = np.pad(log_img, pad_width=1, mode='constant', constant_values=0)
    return log_img

#### Haralick Texture

In [27]:
# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(image).mean(axis=0)
    # return the result
    return haralick

#### Hu Moments

In [28]:
# hu moments (similar to zern)
def fd_hu_moments(image):
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

#### Aspect Ratio

In [29]:
# Calculates the aspect ratio of the min bounding box of a nuclei
def fd_aspect_ratio(cnt):
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(w)/h
    return aspect_ratio

#### Solidity

In [30]:
# Calculates the extent a shape is concave or convex 
def fd_solidity(cnt):
    area = cv2.contourArea(cnt)
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull)
    solid = float(area)/hull_area
    return solid

#### Gabor Wavelets

In [31]:
# Calculate GaborWavelet descriptor
def fd_GaborWavelet(img):
    filters = []
    ksize = 31
    for theta in np.arange(0, np.pi, np.pi / 16):
        kern = cv2.getGaborKernel((ksize, ksize), 4.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    kern /= 1.5 * kern.sum()
    filters.append(kern)
    
    accum = np.zeros_like(img)
    for kern in filters:
        fimg = cv2.filter2D(img, cv2.CV_8UC3, kern)
    np.maximum(accum, fimg, accum)
    return accum

#### Extent

In [32]:

# Calculates the ratio between the nuclei's contour and bounding rectangle
def fd_extent(cnt):
    area = cv2.contourArea(cnt)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = w*h
    extent = float(area)/rect_area
    return extent

#### SIFT

In [33]:
# Calculate SIFT descriptors
def fd_SIFT(img):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(rot_single_nuclei, None)
    return des




#### Histogram of Oriented Gradients

In [34]:
# Histogram of oriented Gradients 
def fd_HOG(img):
    (H, hogImage) = hog(img, orientations=9, pixels_per_cell=(8, 8),
                                cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2-Hys",
                                visualize=True)
    hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
    hogImage = hogImage.astype("uint8")

    return hogImage



#### Equivalent Diameter

In [35]:

# Calculated the Equivalent Diameter of min circle packing
def fd_Equi_diameter(cnt):
    area = cv2.contourArea(cnt)
    equi_diameter = np.sqrt(4*area/np.pi)
    return int(round(equi_diameter))

#### Roundness

In [36]:
# Calculates the roundness of a contour
def fd_roundness(cnt):
    moments = cv2.moments(cnt)
    length = cv2.arcLength(cnt, True)
    k = (length * length) / (moments['m00'] * 4 * np.pi)
    return k

#### Liner Binary Patterns

In [37]:
# Calculate LBP descriptors
def fd_linearbinarypattern(img):
    height, width = img.shape
    img_lbp = np.zeros((height, width, 3), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
            img_lbp[i, j] = lbp_calculated_pixel(img, i, j)
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    hist_lbp = [int(i) for i in hist_lbp]
    return hist_lbp   

def lbp_calculated_pixel(img, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4
    '''
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x - 1, y + 1))  # top_right
    val_ar.append(get_pixel(img, center, x, y + 1))  # right
    val_ar.append(get_pixel(img, center, x + 1, y + 1))  # bottom_right
    val_ar.append(get_pixel(img, center, x + 1, y))  # bottom
    val_ar.append(get_pixel(img, center, x + 1, y - 1))  # bottom_left
    val_ar.append(get_pixel(img, center, x, y - 1))  # left
    val_ar.append(get_pixel(img, center, x - 1, y - 1))  # top_left
    val_ar.append(get_pixel(img, center, x - 1, y))  # top

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

#### Store ND Features

In [38]:

# Store ND array to a dictionary
def StoreNDArray(dict,featurename,data):
    if data is not None:
        if not type(data) == list:
            data = data.flatten()
        for idx,val in enumerate(data):
            dict[featurename+str(idx)] = val
    else:
        dict[featurename + str(0)] = 0


def StoreHaralickFeature(dict,data):
    haralick_featres = ["ang second moment","contrast","correlation","sum of squares: varience", "inverse Diff moment",
                        "sum average","sum varience","sum entropy","entropy", "diff varience","dif entropy","measure of corro 1", "measure of corro 2"]
    for idx,val in enumerate(data):
        dict[haralick_featres[idx]] = val


### Nuclei Segmentation

In [39]:
# Given the nuclei image channel, find nuclei
def detect_NUCLEI(r):
    
    # Gamma correction   
    r = adjust_gamma(r, gamma=1.5)

    # Otsu's thresholding
    ret2, th2 = cv2.threshold(r, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # These contours are then filled using mathematical morphology.
    fill_masks = ndi.binary_fill_holes(th2)

    # Small spurious objects are easily removed by setting a minimum size for valid objects.
    cleaned_image = morphology.remove_small_objects(fill_masks, 150)
    
    # Perform Distance Transform     
    D = ndi.distance_transform_edt(cleaned_image)
    localMax = peak_local_max(D, indices=False, min_distance=20, labels=cleaned_image)
    
    # perform a connected component analysis on the local peaks,
    # using 8-connectivity, then appy the Watershed algorithm
    markers = ndi.label(localMax, structure=np.ones((3, 3)))[0]
    labels = watershed(-D, markers, mask=cleaned_image)
    print("[INFO] {} unique segments found".format(len(np.unique(labels)) - 1))

    # Un comment to bypass watershed
    # labels, num = ndi.label(cleaned_image, structure=np.ones((3, 3)))

    # remove edge nuclei
    labels_image = clear_border(labels)
    
    # recount labels
    number_regions = np.delete(np.unique(labels_image), 0)

    return labels_image, number_regions



### Foci Detection

In [40]:
# Adaptive median filter of an image
def adaptive_median_filter(img,sMax):
    newimg = img.copy()
    height, width = img.shape[:2]
    filterSize = 3
    borderSize = sMax // 2
    imgMax = img[(0, 0)]
    mid = (filterSize * filterSize) // 2
    for i in range(width):
        for j in range(height):
            if (imgMax < img[j,i]):
                imgMax = img[j,i]

    for i in range(borderSize, width - borderSize):
        for j in range(borderSize, height - borderSize):
            members = [imgMax] * (sMax * sMax)
            filterSize = 3
            zxy = img[j,i]
            result = zxy
            while (filterSize <= sMax):
                borderS = filterSize // 2
                for k in range(filterSize):
                    for t in range(filterSize):
                        members[k * filterSize + t] = img[j + t - borderS,i + k - borderS]
                        # print(members[k*filterSize+t])
                members.sort()
                med = (filterSize * filterSize) // 2
                zmin = members[0]
                zmax = members[(filterSize - 1) * (filterSize + 1)]
                zmed = members[med]
                if (zmed < zmax and zmed > zmin):
                    if (zxy > zmin and zxy < zmax):
                        result = zxy
                    else:
                        result = zmed
                    break
                else:
                    filterSize += 2

            newimg[j,i] = result
    return newimg

# produce a top_hat transform from a cell
def top_hat_transform(img,kernel_size):
    # Structuring element
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    # Apply the top hat transform
    tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)

    return tophat

# otsu threshold of the image
def find_otsu_t(img):
    t, thresh_img = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return t, thresh_img

# hmax transform of the image
def h_max_transform(img,h):
    h_maxima = extrema.h_maxima(img, h)
    label_h_maxima = label(h_maxima)
    label_h_maxima[label_h_maxima>0] = 255
    return label_h_maxima

# Detect the foci of a nuclei, given the green channel
def detect_FOCI(g):
    # Dehaze
    dehazed = dehaze(g)

    # Adaptive median filter remove noise     
    for i in range(7, 1, -2):
        g = adaptive_median_filter(g,i)

    # Top hat transform to detect posible foci locations    
    top = top_hat_transform(dehazed,25)
    
    # Convert to int     
    top = np.uint8(top)

    # Detect Foci Regions by Threshold 
    t, mask = find_otsu_t(top)

    # Get only the object by overylaying the mask
    fig = cv2.bitwise_or(top, top, mask=mask)

    # Get the extrema and label the foci  
    labeled = h_max_transform(fig,t)

    return labeled, dehazed


### Labeling

In [41]:
# Detemine class based on foci
def Classify_Image(foci_image):
    b, g, r = cv2.split(foci_image)
    # If anything is in the blue channel then it is labeled as a foci
    if 255 in b:
        return "Damaged"
    return "Healthy"



### Dataset Generation

In [57]:
# Get Frames in the dataset
Frames_list = os.listdir(Dataset_location)
Frames_list.sort()

# Nuclei tracker over all nuclei
nuclei_count = 0

# Report Dictionary
classdict = {}


# for each frame in the dataset...
for frame in Frames_list:
    frame_img_location = os.path.join(Dataset_location,frame)
    print("Generating Nuclei Crops for: " + frame_img_location)

    # Read the image and split by channel
    data_array = cv2.imread(frame_img_location, 1)
    b, g, r = cv2.split(data_array)

    '''
    NOTE: 
    
    "labeled" means that each nuclei is given a 
    unique identifier [1-255] and background is 0
    
    '''
############### Detect FOCI #########################

    # obtain the foci locations
    foci_labeled, dehazed = detect_FOCI(g)

############### Detect Nuclei #########################

    # obtain the nuclei locations
    labeled_cells,num = detect_NUCLEI(r)

    

    # make an image with all GFP, nuclei labels, and foci labels,
    detected_nuclei = labeled_cells.copy()
    detected_nuclei[detected_nuclei>0]=255
    
    # height, width, layers of an image
    height, width = labeled_cells.shape

    all_labels = np.zeros((height, width, 3), dtype="uint8")  # matrix of zeros (black)
    all_labels[:, :, 0] = foci_labeled
    all_labels[:, :, 1] = dehazed
    all_labels[:, :, 2] = detected_nuclei

    # iterate through each nuclei
    for i in num:
                
############### Mask Segmentation #########################
        
        # obtain a mask of the nuclei
        mask = np.where(labeled_cells == i, 255, 0)
    
        '''
        crop the individual cell
        given a mask and the OG image, 
        overlay mask onto the OG image 
        and return the cropped segmented object
        '''
       
        # convert into unsigned int
        mask = np.uint8(mask)

        # overlay mask with the original image
        single_nuclei = cv2.bitwise_and(r, r, mask=mask)

        # obtain contours of the nuclei
        contours, hierarchy = cv2.findContours(single_nuclei, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        if not len(contours) == 0:
            # obtain max contour area
            cnt = max(contours, key=cv2.contourArea)

            # calc bounding box
            x, y, w, h = cv2.boundingRect(cnt)

            # Crop the 3 layer channels
            single_nuclei_foci = cv2.bitwise_and(all_labels[:, :, 0], all_labels[:, :, 0], mask=mask)
            single_nuclei_green_channel = cv2.bitwise_and(all_labels[:, :, 1], all_labels[:, :, 1], mask=mask)

            segmented = np.zeros((height, width, 3), dtype="uint8")  # matrix of zeros (black)

            # make overlay into 3 channels (bgr)
            segmented[:, :, 0] = single_nuclei_foci
            segmented[:, :, 1] = single_nuclei_green_channel
            segmented[:, :, 2] = single_nuclei

            # crop the image based on bounding box
            crop_img = segmented[y - 1:y + h + 1, x - 1:x + w + 1]
            mask = mask[y - 1:y + h + 1, x - 1:x + w + 1]
            crop_img = cv2.bitwise_and(crop_img, crop_img, mask=mask)

            # Increment nuclei count            
            nuclei_count = nuclei_count+1
            
            
            
############### Normalize Nuclei #########################
           
            # Get just the red channel         
            single_nuclei = crop_img[:,:,2]
    
            # Obtain Axis Features         
            x, y, minor, major, maj_angle, min_angle, ecc = get_axis(cnt)
            
            # Rotate the nuclei             
            rot_single_nuclei = rotate_bound(single_nuclei, -maj_angle)
            
            # Pad inamge to 200x200            
            rot_single_nuclei = padImage(rot_single_nuclei, 150, 150)

            # Write nuclei to file             
            crop_loc = "./Crops/" + str(nuclei_count) + ".tif"
            cv2.imwrite(crop_loc, rot_single_nuclei)
            
            # Upscale image       
            rot_single_nuclei = bilinear_upscale(rot_single_nuclei,rate=2)
            cv2.imwrite(crop_loc, rot_single_nuclei)

            ############### Feature Extraction #########################

            print("Obtaining Features for nuclei " + str(nuclei_count))

            classification = Classify_Image(crop_img)
            asp = fd_aspect_ratio(cnt)
            ext = fd_extent(cnt)
            sold = fd_solidity(cnt)
            diam = fd_Equi_diameter(cnt)
            roundness_feature = fd_roundness(cnt)
            zern = fd_Zern(rot_single_nuclei, int(math.floor(diam/2)))
            har = fd_haralick(rot_single_nuclei)
            hu = fd_hu_moments(rot_single_nuclei)
            linear_hist = fd_linearbinarypattern(rot_single_nuclei)
            fd = fd_HOG(rot_single_nuclei)
#             You need to do some special installation for SIFT
#             des = fd_SIFT(rot_single_nuclei)
            log = fd_LoG(rot_single_nuclei)
            Gabor = fd_GaborWavelet(rot_single_nuclei)
            
############### Write to file #########################


            classdict[crop_loc] = {}
            
            # 1D descriptors           
            classdict[crop_loc]["Original Frame"] = frame_img_location
            classdict[crop_loc]["Cropped Frame"] = crop_loc
            classdict[crop_loc]["True Classification"] = classification
            classdict[crop_loc]["Centroid_x"] = x
            classdict[crop_loc]["Centroid_y"] = y
            classdict[crop_loc]["Major Axis"] = major
            classdict[crop_loc]["Minor Axis"] = minor
            classdict[crop_loc]["Eccentricity"] = ecc
            classdict[crop_loc]["Aspect Ratio"] = asp
            classdict[crop_loc]["Solidity"] = sold
            classdict[crop_loc]["Equivalent Diameter"] = diam
            classdict[crop_loc]["Roundness"] = roundness_feature
            classdict[crop_loc]["Extent"] = ext
            
            # 2D descriptors             
            
            StoreNDArray(classdict[crop_loc], "Zernlike Moments", zern)
 
            StoreHaralickFeature(classdict[crop_loc],har)

            # classdict[crop_loc]["Hu Moments"] = hu
            StoreNDArray(classdict[crop_loc], "Hu Moments", hu)
            
            StoreNDArray(classdict[crop_loc], "Linear Binary Patterns", linear_hist)

            
            HOG_loc = "./HOG/" + str(nuclei_count)+".tif"
            cv2.imwrite(HOG_loc, fd)
            classdict[crop_loc]["HOG"] = HOG_loc

#             StoreNDArray(classdict[crop_loc], "SIFT", des)

            LOG_loc = "./LOG/" + str(nuclei_count) + ".tif"
            cv2.imwrite(LOG_loc, log)
            classdict[crop_loc]["Laplace of Gaussian"] = LOG_loc

            Gabor_loc = "./Gabor/" + str(nuclei_count) + ".tif"
            cv2.imwrite(Gabor_loc,Gabor)
            classdict[crop_loc]["Gabor Wavelet"] = Gabor_loc
            
        
print("Writing to file...")        
dataframe = pd.DataFrame.from_dict(classdict).T
dataframe.to_csv('dataset_report.csv', index=False)
print("Done!")


Generating Nuclei Crops for: ./Dataset/WellC3_Seq0010_WellC3_Seq0010_T01_XY1_RGB.tif
[INFO] 13 unique segments found
Obtaining Features for nuclei 1
Obtaining Features for nuclei 2
Obtaining Features for nuclei 3
Obtaining Features for nuclei 4
Obtaining Features for nuclei 5
Obtaining Features for nuclei 6
Obtaining Features for nuclei 7
Obtaining Features for nuclei 8
Obtaining Features for nuclei 9
Obtaining Features for nuclei 10
Obtaining Features for nuclei 11
Obtaining Features for nuclei 12
Obtaining Features for nuclei 13
Generating Nuclei Crops for: ./Dataset/WellC3_Seq0010_WellC3_Seq0010_T02_XY1_RGB.tif
[INFO] 11 unique segments found
Obtaining Features for nuclei 14
Obtaining Features for nuclei 15
Obtaining Features for nuclei 16
Obtaining Features for nuclei 17
Obtaining Features for nuclei 18
Obtaining Features for nuclei 19
Obtaining Features for nuclei 20
Obtaining Features for nuclei 21
Obtaining Features for nuclei 22
Obtaining Features for nuclei 23
Obtaining Featur

Obtaining Features for nuclei 187
Obtaining Features for nuclei 188
Obtaining Features for nuclei 189
Obtaining Features for nuclei 190
Obtaining Features for nuclei 191
Obtaining Features for nuclei 192
Obtaining Features for nuclei 193
Obtaining Features for nuclei 194
Obtaining Features for nuclei 195
Generating Nuclei Crops for: ./Dataset/WellC3_Seq0010_WellC3_Seq0010_T19_XY1_RGB.tif
[INFO] 11 unique segments found
Obtaining Features for nuclei 196
Obtaining Features for nuclei 197
Obtaining Features for nuclei 198
Obtaining Features for nuclei 199
Obtaining Features for nuclei 200
Obtaining Features for nuclei 201
Obtaining Features for nuclei 202
Obtaining Features for nuclei 203
Obtaining Features for nuclei 204
Obtaining Features for nuclei 205
Obtaining Features for nuclei 206
Generating Nuclei Crops for: ./Dataset/WellC3_Seq0010_WellC3_Seq0010_T20_XY1_RGB.tif
[INFO] 11 unique segments found
Obtaining Features for nuclei 207
Obtaining Features for nuclei 208
Obtaining Features

### Feature Reduction

In [58]:
dataset = pd.read_csv("./dataset_report.csv")

meta_headers = ['Cropped Frame','Original Frame', "HOG","Laplace of Gaussian","Gabor Wavelet","Centroid_x","Centroid_y"]
meta_data = dataset.filter(meta_headers, axis=1)

# Mix and matrch what features are important by removing specific features
# SIFT = dataset.filter(regex=("SIFT.*"))
# LBP = dataset.filter(regex=("Linear Binary Patterns.*"))
# dataset.drop(SIFT, axis=1, inplace=True)
# dataset.drop(LBP, axis=1, inplace=True)
# ZLM = dataset.filter(regex=("Zernlike Moments.*"))
# dataset.drop(ZLM, axis=1, inplace=True)

dataset.drop(meta_headers, axis=1, inplace=True)
dataset.drop(labels=['True Classification'], axis=1,inplace = True)

dataset= dataset.replace("Healthy",-1)
dataset = dataset.replace("Damaged",1)

dataset.fillna(0,inplace=True)


# Print top features based on selection metrics
print(pymrmr.mRMR(dataset, 'MIQ',10))
print(pymrmr.mRMR(dataset, 'MID',10))


['sum varience', 'Roundness', 'Linear Binary Patterns180', 'Linear Binary Patterns169', 'Linear Binary Patterns166', 'Linear Binary Patterns74', 'Linear Binary Patterns73', 'Linear Binary Patterns208', 'Linear Binary Patterns15', 'sum of squares: varience']
['sum varience', 'Solidity', 'Roundness', 'Zernlike Moments4', 'Extent', 'Zernlike Moments11', 'Linear Binary Patterns180', 'Linear Binary Patterns169', 'Linear Binary Patterns166', 'Linear Binary Patterns15']


### Classification
Play around with the features you want to use and which ones have better classification power

#### Metrics

In [59]:
def calcMetrics(confusion_matrix):
    TP = confusion_matrix[0,0]
    FP = confusion_matrix[0,1]
    FN = confusion_matrix[1,0]
    TN = confusion_matrix[1,1]

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

    if TP == 0:
        fscore = 0
    else:
        fscore = 2*((precision*recall)/(precision+recall))

    # false negative rate
    FNR = FN/(TP+FN)

    # false positive rate
    FDR = FP/(TP+FP)

    # specificity
    SPC = TN/(FN+TN)

    # accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)

    return fscore,FNR,FDR,SPC,ACC

#### Random Forest

In [60]:

dataset = pd.read_csv("./dataset_report.csv")


meta_headers = ['Cropped Frame','Original Frame', "HOG","Laplace of Gaussian","Gabor Wavelet","Centroid_x","Centroid_y"]

meta_data = dataset.filter(meta_headers, axis=1)
dataset.drop(meta_headers, axis=1, inplace=True)

dataset= dataset.replace("Healthy",1)
dataset = dataset.replace("Damaged",0)

dataset = dataset.fillna(0)


# move classes to first column
mid = dataset['True Classification']
# print(mid)
dataset.drop(labels=['True Classification'], axis=1,inplace = True)

head = np.array(dataset.columns.values)

dataset.insert(0, 'True Classification', mid)

dataset = np.array(dataset)
# Shuffle the data so that the K folds are not to be influenced by the original frame
np.random.shuffle(dataset)

X = dataset[:, 1:]
y = dataset[:, 0]

max_features = min(X.shape[1],X.shape[0])
splits=10
kf = KFold(n_splits=splits)
accuracy_sum = 0
fscore_sum = 0
FNR_sum = 0
FDR_sum = 0
SPC_sum = 0

for train, test in kf.split(dataset):
    X_train = dataset[train][:,1:]
    y_train = dataset[train][:,0]

    X_test = dataset[test][:,1:]
    y_test = dataset[test][:,0]


    lab_enc = preprocessing.LabelEncoder()
    training_scores_encoded = lab_enc.fit_transform(y_train)

    regressor = RandomForestClassifier(n_estimators=100, random_state=2, max_depth=10, criterion='gini',max_features=max_features)
    regressor.fit(X_train, training_scores_encoded)
    y_pred = regressor.predict(X_test)



    # appendDFToCSV_void(importances.T, report_location,head)

    cn_matrix = confusion_matrix(y_test, y_pred)
    print(cn_matrix)
    fscore, FNR, FDR, SPC, ACC = calcMetrics(cn_matrix)

    accuracy_sum = accuracy_sum + ACC
    fscore_sum = fscore_sum + fscore
    FNR_sum = FNR_sum + FNR
    FDR_sum = FDR_sum + FDR
    SPC_sum = SPC_sum + SPC


accuracy_average = accuracy_sum/splits
fscore_avg = fscore_sum/splits
FNR_avg = FNR_sum/splits
FDR_avg = FDR_sum/splits
SPC_avg = SPC_sum/splits

print("Average F-score: " + str(fscore_avg))
print("Average Accuracy: " + str(accuracy_average))
print("Average False Negative Rate: " + str(FNR_avg))
print("Average Specificity: " + str(SPC_avg))
print("Average False Discovery Rate: " + str(FDR_avg))

[[ 3  4]
 [ 0 18]]
[[ 5  0]
 [ 0 20]]
[[ 2  3]
 [ 0 20]]
[[ 4  0]
 [ 0 21]]
[[ 2  2]
 [ 0 21]]
[[ 3  0]
 [ 1 21]]
[[ 3  1]
 [ 1 20]]
[[ 1  1]
 [ 0 22]]
[[ 1  0]
 [ 2 21]]
[[ 4  1]
 [ 0 19]]
Average F-score: 0.7500793650793651
Average Accuracy: 0.9353333333333333
Average False Negative Rate: 0.11666666666666665
Average Specificity: 0.9819969885187276
Average False Discovery Rate: 0.2621428571428571


#### ResNet

In [68]:

input_shape=(32, 32,1)
num_classes=2
class_weight = {0: 3,1: 1.}
splits=10


# compile the CNN here
def compileCNN():
    x = keras.layers.Input(input_shape)
    model = keras_resnet.models.ResNet50(x, classes=num_classes)
    model.compile("adam","binary_crossentropy", ["accuracy",f1_m],)
    return model

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# Given the ing link data
# read each image and resize to 32x32
def collectImageData(data):
    for feature in range(0, 4):
        for idx, file in enumerate(data[:, feature]):
#             print(file)
            img = cv2.imread(file,0)
            resized = resize(img, input_shape)
            data[idx, feature] = resized
    return data



def show_traingraph(history):
    # list all data in history
    # print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Run the CNN
def runCNN(data):

    kf = KFold(n_splits=splits)

    Prob_dataset = []

    for feature in range(0, 4):

        # compute new model for each feature
        model = compileCNN()

        accuracy_sum = 0
        fscore_sum = 0
        FNR_sum = 0
        FDR_sum = 0
        SPC_sum = 0

        Probability_as_Feature = []

        for train, test in kf.split(data):
            train_size = len(train)
            test_size = len(test)

            x_train = np.empty((train_size,32, 32,1))
            y_train = to_categorical(data[train,4],num_classes=2)

            # compute class weight
            # class_weight[0] =  train_size / sum(data[train, 4])

            y_test = data[test, 4]

            for i in range(train_size):
                x_train[i] = data[i,feature]/255

            x_test = np.empty((test_size, 32, 32,1))
            for i in range(test_size):
                x_test[i] = data[i, feature]/255

            print("Training... ")
            hist = model.fit(x_train, y_train, batch_size=100, epochs=10, validation_split=0.33, shuffle= True, class_weight=class_weight,verbose=1)
            # show_traingraph(hist)

            probs = model.predict(x_test, batch_size=64)


            # test sets are done in sequential order, so we can do this here
            for prob in probs:
                Probability_as_Feature.append(prob[0])


            y_pred_bool = np.argmax(probs, axis=1).astype(int)

            y_actu = pd.Series(y_test, name='Actual')
            y_pred = pd.Series(y_pred_bool, name='Predicted')

            df_confusion = pd.crosstab(y_actu, y_pred)
            print(df_confusion)


            #Check if confusion matrix is 2x2
            empty_class = [0, 0]
            if '0' not in df_confusion.columns:
                df_confusion['0'] = empty_class
            if '1' not in df_confusion.columns:
                df_confusion['1'] = empty_class

            # Calc all metrics
            fscore, FNR, FDR, SPC, ACC = calcMetrics(np.asarray(df_confusion))
            accuracy_sum = accuracy_sum + ACC
            fscore_sum = fscore_sum + fscore
            FNR_sum = FNR_sum + FNR
            FDR_sum = FDR_sum + FDR
            SPC_sum = SPC_sum + SPC
        accuracy_average = accuracy_sum / splits
        fscore_avg = fscore_sum / splits
        FNR_avg = FNR_sum / splits
        FDR_avg = FDR_sum / splits
        SPC_avg = SPC_sum / splits


        print("\nFeature: " + str(feature))
        print("Average F-score: " + str(fscore_avg))
        print("Average Accuracy: " + str(accuracy_average))
        print("Average False Negative Rate: " + str(FNR_avg))
        print("Average Selectivity: " + str(SPC_avg))
        print("Average False Discovery Rate: " + str(FDR_avg)+"\n")

        stat_array = [feature,fscore_avg,accuracy_average,FNR_avg,SPC_avg,FDR_avg]


        with open("resnet_test_"+str(feature)+".csv", "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(stat_array)

        # print(Probability_as_Feature)
        Prob_dataset.append(Probability_as_Feature)

    return Prob_dataset




# NOTE we do not need to scale the features as we are using a decision based algorithm

dataset = pd.read_csv("./dataset_report.csv")
dataset= dataset.replace("Healthy",1)
dataset = dataset.replace("Damaged",0)

meta_headers = ['Cropped Frame','Original Frame', "HOG","Laplace of Gaussian","Gabor Wavelet","SIFT","Centroid_x","Centroid_y"]
CNN_HEADERS = ['Cropped Frame',"HOG","Laplace of Gaussian","Gabor Wavelet","True Classification"]
CNN_data = dataset[CNN_HEADERS]
CNN_data['Cropped Frame Location'] = dataset['Cropped Frame'].values
CNN_data = np.array(CNN_data)

# Shuffle the data so that the K folds are not to be influenced by the original frame
# np.random.shuffle(CNN_data)

shuffled_cropped_image_locations = CNN_data[:,5]
# print(shuffled_cropped_image_locations)

CNN_imagedata = collectImageData(CNN_data)


Prob_dataset_as_lists = runCNN(CNN_imagedata)
# print(Prob_dataset_as_lists)
# Create empty DF

Join_headers = ['Cropped Frame','Cropped Frame_prob',"HOG_prob","Laplace of Gaussian_prob","Gabor Wavelet_prob"]
my_df = pd.DataFrame(columns = Join_headers)
my_df['Cropped Frame'] = shuffled_cropped_image_locations
my_df['Cropped Frame_prob'] = Prob_dataset_as_lists[0]
my_df['HOG_prob'] = Prob_dataset_as_lists[1]
my_df['Laplace of Gaussian_prob'] = Prob_dataset_as_lists[2]
my_df['Gabor Wavelet_prob'] = Prob_dataset_as_lists[3]
# probs_as_df = pd.DataFrame(data=Prob_dataset,columns=CNN_HEADERS)

dataset = pd.merge(my_df,dataset, on=['Cropped Frame'])

print(dataset['Cropped Frame_prob'])


dataset.to_csv('Dataset_ensable_new.csv',index=False)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['./Crops/1.tif' './Crops/2.tif' './Crops/3.tif' './Crops/4.tif'
 './Crops/5.tif' './Crops/6.tif' './Crops/7.tif' './Crops/8.tif'
 './Crops/9.tif' './Crops/10.tif' './Crops/11.tif' './Crops/12.tif'
 './Crops/13.tif' './Crops/14.tif' './Crops/15.tif' './Crops/16.tif'
 './Crops/17.tif' './Crops/18.tif' './Crops/19.tif' './Crops/20.tif'
 './Crops/21.tif' './Crops/22.tif' './Crops/23.tif' './Crops/24.tif'
 './Crops/25.tif' './Crops/26.tif' './Crops/27.tif' './Crops/28.tif'
 './Crops/29.tif' './Crops/30.tif' './Crops/31.tif' './Crops/32.tif'
 './Crops/33.tif' './Crops/34.tif' './Crops/35.tif' './Crops/36.tif'
 './Crops/37.tif' './Crops/38.tif' './Crops/39.tif' './Crops/40.tif'
 './Crops/41.tif' './Crops/42.tif' './Crops/43.tif' './Crops/44.tif'
 './Crops/45.tif' './Crops/46.tif' './Crops/47.tif' './Crops/48.tif'
 './Crops/49.tif' './Crops/50.tif' './Crops/51.tif' './Crops/52.tif'
 './Crops/53.tif' './Crops/54.tif' './Crops/55.tif' './Crops/56.tif'
 './Crops/57.tif' './Crops/58.tif' './Crops

./HOG/21.tif
./HOG/22.tif
./HOG/23.tif
./HOG/24.tif
./HOG/25.tif
./HOG/26.tif
./HOG/27.tif
./HOG/28.tif
./HOG/29.tif
./HOG/30.tif
./HOG/31.tif
./HOG/32.tif
./HOG/33.tif
./HOG/34.tif
./HOG/35.tif
./HOG/36.tif
./HOG/37.tif
./HOG/38.tif
./HOG/39.tif
./HOG/40.tif
./HOG/41.tif
./HOG/42.tif
./HOG/43.tif
./HOG/44.tif
./HOG/45.tif
./HOG/46.tif
./HOG/47.tif
./HOG/48.tif
./HOG/49.tif
./HOG/50.tif
./HOG/51.tif
./HOG/52.tif
./HOG/53.tif
./HOG/54.tif
./HOG/55.tif
./HOG/56.tif
./HOG/57.tif
./HOG/58.tif
./HOG/59.tif
./HOG/60.tif
./HOG/61.tif
./HOG/62.tif
./HOG/63.tif
./HOG/64.tif
./HOG/65.tif
./HOG/66.tif
./HOG/67.tif
./HOG/68.tif
./HOG/69.tif
./HOG/70.tif
./HOG/71.tif
./HOG/72.tif
./HOG/73.tif
./HOG/74.tif
./HOG/75.tif
./HOG/76.tif
./HOG/77.tif
./HOG/78.tif
./HOG/79.tif
./HOG/80.tif
./HOG/81.tif
./HOG/82.tif
./HOG/83.tif
./HOG/84.tif
./HOG/85.tif
./HOG/86.tif
./HOG/87.tif
./HOG/88.tif
./HOG/89.tif
./HOG/90.tif
./HOG/91.tif
./HOG/92.tif
./HOG/93.tif
./HOG/94.tif
./HOG/95.tif
./HOG/96.tif
./HOG/97.tif

./Gabor/127.tif
./Gabor/128.tif
./Gabor/129.tif
./Gabor/130.tif
./Gabor/131.tif
./Gabor/132.tif
./Gabor/133.tif
./Gabor/134.tif
./Gabor/135.tif
./Gabor/136.tif
./Gabor/137.tif
./Gabor/138.tif
./Gabor/139.tif
./Gabor/140.tif
./Gabor/141.tif
./Gabor/142.tif
./Gabor/143.tif
./Gabor/144.tif
./Gabor/145.tif
./Gabor/146.tif
./Gabor/147.tif
./Gabor/148.tif
./Gabor/149.tif
./Gabor/150.tif
./Gabor/151.tif
./Gabor/152.tif
./Gabor/153.tif
./Gabor/154.tif
./Gabor/155.tif
./Gabor/156.tif
./Gabor/157.tif
./Gabor/158.tif
./Gabor/159.tif
./Gabor/160.tif
./Gabor/161.tif
./Gabor/162.tif
./Gabor/163.tif
./Gabor/164.tif
./Gabor/165.tif
./Gabor/166.tif
./Gabor/167.tif
./Gabor/168.tif
./Gabor/169.tif
./Gabor/170.tif
./Gabor/171.tif
./Gabor/172.tif
./Gabor/173.tif
./Gabor/174.tif
./Gabor/175.tif
./Gabor/176.tif
./Gabor/177.tif
./Gabor/178.tif
./Gabor/179.tif
./Gabor/180.tif
./Gabor/181.tif
./Gabor/182.tif
./Gabor/183.tif
./Gabor/184.tif
./Gabor/185.tif
./Gabor/186.tif
./Gabor/187.tif
./Gabor/188.tif
./Gabor/

KeyboardInterrupt: 

#### Ensamble

In [ ]:

splits=10
kf = KFold(n_splits=splits)
accuracy_sum = 0
fscore_sum = 0
FNR_sum = 0
FDR_sum = 0
SPC_sum = 0

report_location = "featureselection_2d.csv"
if os.path.isfile(report_location):
    os.remove(report_location)
# NOTE we do not need to scale the features as we are using a decision based algorithm


dataset = pd.read_csv("./Dataset_ensable_new.csv")

clone = dataset.copy(deep=True)


dataset= dataset.replace("Healthy",1)
dataset = dataset.replace("Damaged",0)

meta_headers = ['Cropped Frame','Original Frame', "HOG","Laplace of Gaussian","Gabor Wavelet","Centroid_x","Centroid_y"]
SIFT = dataset.filter(regex=("SIFT.*"))


LBP = dataset.filter(regex=("Linear Binary Patterns.*"))
dataset.drop(SIFT, axis=1, inplace=True)
dataset.drop(LBP, axis=1, inplace=True)
ZLM = dataset.filter(regex=("Zernlike Moments.*"))
dataset.drop(ZLM, axis=1, inplace=True)
# meta_data = dataset.filter(meta_headers, axis=1)



meta_data = dataset.filter(meta_headers, axis=1)
dataset.drop(meta_headers, axis=1, inplace=True)



dataset = dataset.fillna(0)


# move classes to first column
mid = dataset['True Classification']
# print(mid)
dataset.drop(labels=['True Classification'], axis=1,inplace = True)

head = np.array(dataset.columns.values)

dataset.insert(0, 'True Classification', mid)

from sklearn.utils import shuffle
df = shuffle(dataset)
# df.reset_index(inplace=True, drop=True)


dataset = np.array(df)

pred_classifications = []

# Shuffle the data so that the K folds are not to be influenced by the original frame
# np.random.shuffle(dataset)

X = dataset[:, 1:]
y = dataset[:, 0]

max_features = min(X.shape[1],X.shape[0])

import matplotlib.pyplot as plt
for train, test in kf.split(dataset):
    X_train = dataset[train][:,1:]

    X_test = dataset[test][:,1:]
    y_train = dataset[train][:,0]

    y_test = dataset[test][:,0]


    lab_enc = preprocessing.LabelEncoder()
    training_scores_encoded = lab_enc.fit_transform(y_train)


    regressor = RandomForestClassifier(n_estimators=100, random_state=2, max_depth=10, criterion='gini', max_features=max_features,)
    regressor.fit(X_train, training_scores_encoded)
    y_pred = regressor.predict(X_test)


    for prediction in y_pred:
        pred_classifications.append(prediction)

    '''
    importances = regressor.feature_importances_
    std = np.std([tree.feature_importances_ for tree in regressor.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, head[indices[f]], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
            color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()
    
    '''




    conf_mat = confusion_matrix(y_test, y_pred)

    fscore, FNR, FDR, SPC, ACC = calcMetrics(conf_mat)


    accuracy_sum = accuracy_sum + ACC
    fscore_sum = fscore_sum + fscore
    FNR_sum = FNR_sum + FNR
    FDR_sum = FDR_sum + FDR
    SPC_sum = SPC_sum + SPC


accuracy_average = accuracy_sum/splits
fscore_avg = fscore_sum/splits
FNR_avg = FNR_sum/splits
FDR_avg = FDR_sum/splits
SPC_avg = SPC_sum/splits

print("Average F-score: " + str(fscore_avg))
print("Average Accuracy: " + str(accuracy_average))
print("Average False Negative Rate: " + str(FNR_avg))
print("Average Specificity: " + str(SPC_avg))
print("Average False Discovery Rate: " + str(FDR_avg))


# dataset = pd.read_csv("./Dataset_ensable_new.csv")
clone["Pred Classifications"] = pred_classifications
clone.to_csv("./dataset_ensable_predclasses.csv")


### Feature Analysis

In [ ]:


dataset = pd.read_csv("./dataset_ensable_predclasses.csv")


meta_headers = ['Cropped Frame','Original Frame', "HOG","Laplace of Gaussian","Gabor Wavelet","Centroid_x","Centroid_y"]
SIFT = dataset.filter(regex=("SIFT.*"))
LBP = dataset.filter(regex=("Linear Binary Patterns.*"))
# dataset.drop(SIFT, axis=1, inplace=True)
# dataset.drop(LBP, axis=1, inplace=True)

meta_data = dataset.filter(meta_headers, axis=1)
dataset.drop(meta_headers, axis=1, inplace=True)


dataset= dataset.replace("Healthy",1)
dataset = dataset.replace("Damaged",0)

dataset = dataset.fillna(0)


# move classes to first column
TC = dataset['True Classification']
dataset.drop(labels=['True Classification'], axis=1, inplace = True)

PC = dataset["Pred Classifications"]
dataset.drop(labels=['Pred Classifications'], axis=1, inplace = True)

#convet True classes and predicted classes into 1,2,3,4 based on TP,TN,FP,FN

PC = np.array(PC,dtype=int)
TC = np.array(TC,dtype=int)
color_labels = np.empty(shape=(TC.shape[0],1))

for data_point in range(TC.shape[0]):
    if TC[data_point] == 1 and PC[data_point] == 1:
        # True Positive
        color_labels[data_point] = 0
    elif TC[data_point] == 0 and PC[data_point] == 1:
        # False Negative
        color_labels[data_point] = 1
    elif TC[data_point] == 1 and PC[data_point] == 0:
        # False Positive
        color_labels[data_point] = 2
    else:
        # True Negative
        color_labels[data_point] = 3

print(color_labels)
color_df = dataset.copy(deep=True)
color_df.insert(0, 'Color Classification', color_labels)

color_df = np.array(color_df,dtype=float)
# Initialise the Scaler
scaler = MinMaxScaler()
scaler.fit(color_df)
color_df = scaler.transform(color_df)



# Shuffle the data so that the K folds are not to be influenced by the original frame
np.random.shuffle(color_df)

pca = PCA(n_components=2)

X_train = color_df[:,1:]
X_test = color_df[:,1:]
y_train = color_df[:,0]
y_test = color_df[:,0]

pca.fit(X_train)
X_test = pca.transform(X_test)

# import matplotlib.collections.PathCollection.legend_elements

# print(X_train)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Principal Component 1', fontsize=15)
ax.set_ylabel('Principal Component 2', fontsize=15)
ax.set_title('2 component PCA', fontsize=20)
targets = ['TP', 'FN','FP','TN']
colors = ['g', 'y', 'b','r']


scatter = ax.scatter(X_test[:,0]
           , X_test[:,1]
           , c=y_test
           , alpha= 0.3
           , s=25
           ,cmap= matplotlib.colors.ListedColormap(colors))

# todo make legends

bounds = np.linspace(0,3,4)
cb = plt.colorbar(scatter, spacing='proportional',ticks=bounds)
# cb.set_label('Custom cbar')

# ax.legend(targets[0])
ax.grid()
plt.show()




dataset.insert(0, 'True Classification', TC)

dataset = np.array(dataset,dtype=float)

# Initialise the Scaler
scaler = MinMaxScaler()
scaler.fit(dataset)
dataset = scaler.transform(dataset)



# Shuffle the data so that the K folds are not to be influenced by the original frame
np.random.shuffle(dataset)



pca = PCA(n_components=2)

X_train = dataset[:,1:]
X_test = dataset[:,1:]
y_train = dataset[:,0]
y_test = dataset[:,0]

pca.fit(X_train)
X_test = pca.transform(X_test)


# print(X_train)
fig1 = plt.figure(figsize=(8, 8))
ax1 = fig1.add_subplot(1, 1, 1)
ax1.set_xlabel('Principal Component 1', fontsize=15)
ax1.set_ylabel('Principal Component 2', fontsize=15)
ax1.set_title('2 component PCA', fontsize=20)
targets = ['TP', 'TN']
colors = ['r', 'g']

# for target, color in zip(targets, colors):
scatter = ax1.scatter(X_test[:,0]
           , X_test[:,1]
           , c= y_test
           , s= 25
           , alpha= 0.3
           , cmap= matplotlib.colors.ListedColormap(colors))

bounds = np.linspace(0,1,2)
cb = plt.colorbar(scatter, spacing='proportional',ticks=bounds)
ax1.grid()
plt.show()

In [ ]:


IMAGE_SIZE = 20

def read_and_resizeimage(img_src):
    img = cv2.imread(img_src,0)
    img = cv2.resize(img,(32,32))
    return img

import math

def appendImage(tiny_img,large_image,y_offset,x_offset,pred_class):
    rows = tiny_img.shape[0]
    cols = tiny_img.shape[1]
    for x in range(0, rows-1):
        for y in range(0, cols-1):
            large_image[int(x+x_offset), int(y+y_offset),pred_class+1] = int(tiny_img[x,y])
    return  large_image



def combineImages(df, col=12, rows=12, size=32):
    # Width is constant, Height is defined by how many images it can fit.
    #
    # Num per row =
    height = rows*size
    width = col*size

    img_arr = np.zeros((height, width,3),dtype=int)

    y_offset = 0
    x_offset = 0

    for index, row in df.iterrows():
        print(x_offset)
        print(y_offset)

        img_src = row['Cropped Frame']
        pred_class = abs(row['Pred Classifications']-1)
        img = read_and_resizeimage(img_src)

        img_arr = appendImage(img, img_arr, y_offset, x_offset,pred_class)

        # calc new offset
        # print(index)
        scale = int(math.fmod(index,rows))
        print(scale)
        x_offset += size

        if(x_offset==width):
            y_offset+=size
            x_offset = 0


    return img_arr



# combine images
dataset = pd.read_csv("./dataset_ensable_predclasses.csv")
wanted_headers = ["Cropped Frame","True Classification","Pred Classifications"]
meta_data = dataset.filter(wanted_headers, axis=1)
is_damaged = meta_data['True Classification'] == 0
is_undamaged = meta_data['True Classification'] == 1

# the plot number should ideally be a prefect square
damaged = meta_data[is_damaged].head(144).reset_index()
undamaged = meta_data[is_undamaged].head(144).reset_index()

result = combineImages(undamaged)

# cv2.imwrite("Undamaged.png", result)

result = combineImages(damaged)
cv2.imwrite("Damaged.png", result)